In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

In [2]:
df = pd.read_csv('alpha24_step15Stud.csv')
df.head()

,czas(min),alpha,temp,Ts
0,0,1,"6,50",2
1,15,0,"6,35","2,18"
2,30,0,"6,20","2,19"
3,45,1,"6,05","1,68"
4,60,1,"5,90","1,5"


In [3]:
df['temp'] = df['temp'].apply(lambda x: x.replace(',','.')).astype('float64')
df['Ts'] = df['Ts'].apply(lambda x: x.replace(',','.')).astype('float64')

In [4]:
df.head()

,czas(min),alpha,temp,Ts
0,0,1,6.50,2.00
1,15,0,6.35,2.18
2,30,0,6.20,2.19
3,45,1,6.05,1.68
4,60,1,5.90,1.50


In [5]:
df.rename(columns={"czas(min)": "time", "alpha": "compressor", "temp": 'ambient_temp', "Ts": 'refrigerator_temp'}, inplace=True)
df.set_index('time', inplace=True)
df.head()

,compressor,ambient_temp,refrigerator_temp
time,,,
0,1,6.50,2.00
15,0,6.35,2.18
30,0,6.20,2.19
45,1,6.05,1.68
60,1,5.90,1.50


In [6]:
size = 1000
delay = 100

a = df.iloc[0:size,:].to_numpy().reshape(delay,size*3//delay)
a

array([[ 1.  ,  6.5 ,  2.  , ...,  0.  ,  5.26,  1.82],
       [ 0.  ,  5.23,  1.86, ...,  0.  ,  5.39,  1.48],
       [ 0.  ,  5.3 ,  1.53, ...,  1.  ,  5.99,  0.9 ],
       ...,
       [ 0.  ,  1.97, -1.98, ...,  0.  ,  1.23, -1.93],
       [ 0.  ,  1.2 , -1.92, ...,  0.  ,  2.01, -2.07],
       [ 1.  ,  2.13, -2.17, ...,  1.  ,  4.01, -2.48]])

In [7]:
import random_forest
from random_forest.tree import Tree

random_forest.tree.config.min_split_size = 15
random_forest.tree.config.number_of_predictors_to_draw = 5

In [8]:
myTree = Tree(a)
test_data = df.iloc[1000:1010,:].to_numpy().reshape(30)
print(test_data, myTree.predict(test_data))

[ 1.    4.2  -2.5   1.    4.39 -2.51  1.    4.57 -2.52  1.    4.74 -2.53
  1.    4.9  -2.54  1.    5.05 -2.55  1.    5.18 -2.56  1.    5.3  -2.57
  1.    5.4  -2.57  0.    5.35 -2.35] -2.6727777777777773


In [9]:
print(myTree)


        min_split_size: 30
        number_of_predictors_to_draw: 5
        
        Branch:
            if <= 10_5.28:
                
        Branch:
            if <= 23_-1.54:
                
        Branch:
            if <= 20_-2.33:
                Leaf: -2.6727777777777773
            else:
                Leaf: -1.8605  
        
            else:
                Leaf: -0.7527777777777778  
        
            else:
                
        Branch:
            if <= 8_-0.97:
                Leaf: -1.6763636363636365
            else:
                
        Branch:
            if <= 8_0.13:
                Leaf: -0.19333333333333333
            else:
                Leaf: 0.7300000000000001  
          
          
        
        


In [10]:
from random_forest.forest import Forest
import random_forest
random_forest.forest.config.data_size_for_tree = 150
random_forest.forest.config.number_of_trees = 100


In [11]:


delay = 4
frames = [df.shift(-t) for t in range(delay)]
# small_df = df.iloc[:40]
delayed_df = pd.concat(frames, axis=1).dropna()

small_df = delayed_df.iloc[:50]
small_df.shape


(50, 12)

In [12]:
forest = Forest(delayed_df)
 

In [13]:
out_of_bag  = small_df
print(out_of_bag.shape)
data_size_for_tree = 5
for i in range(10):
    training_data = small_df.sample(data_size_for_tree, replace=True)
    out_of_bag = out_of_bag[~out_of_bag.index.isin(training_data.index)]
# out_of_bag[~out_of_bag.isin(training_data)].dropna()
# out_of_bag.isin(training_data.merge(training_data))
# print(out_of_bag.size)
print(out_of_bag.shape)
training_data.head()

(50, 12)
(24, 12)


,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp
time,,,,,,,,,,,,
735,0,7.99,0.59,0.0,7.97,0.84,0.0,7.94,0.93,0.0,7.90,0.98
90,0,5.59,1.62,0.0,5.44,1.56,0.0,5.30,1.54,0.0,5.26,1.82
135,0,5.26,1.82,0.0,5.23,1.86,0.0,5.21,1.89,1.0,5.20,1.20
255,0,5.60,1.22,0.0,5.49,1.22,0.0,5.39,1.48,0.0,5.30,1.53
315,1,5.39,1.12,0.0,5.49,1.26,0.0,5.59,1.47,0.0,5.70,1.51


out_of_bag.values


In [14]:
print(forest.out_of_bag.shape)
test = forest.out_of_bag.iloc[0].values
print(test)
print(forest.predict(test[:-1]))
# df.shape

(25, 12)
[1.   5.7  1.07 0.   5.6  1.22 0.   5.49 1.22 0.   5.39 1.48]
1.1334219240029655


In [16]:
forest.trees[0]


        min_split_size: 30
        number_of_predictors_to_draw: 5
        
        Branch:
            if <= 8_-0.43:
                
        Branch:
            if <= 8_-1.52:
                
        Branch:
            if <= 5_-2.4:
                Leaf: -2.7421428571428565
            else:
                
        Branch:
            if <= 5_-2.1:
                Leaf: -2.111428571428571
            else:
                Leaf: -1.7741666666666667  
          
        
            else:
                
        Branch:
            if <= 1_3.54:
                Leaf: -1.4460000000000002
            else:
                
        Branch:
            if <= 5_-0.92:
                Leaf: -1.07
            else:
                Leaf: -0.66875  
          
          
        
            else:
                
        Branch:
            if <= 2_0.87:
                
        Branch:
            if <= 8_0.16:
                Leaf: -0.13999999999999999
            else:
               